In [13]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit

from parallel_two_qubit_gate_decomposition import *
from gates_numpy import cnot_gate, fsim_gate, cphase_gate, xy_gate, get_gate_unitary_qiskit



## Decomposition example with a single two-qubit gate 

In [14]:
# CNOT gate in the circuit
my_op = np.matrix([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,0,1],
                   [0,0,1,0]])

In [15]:
test_circuit = QuantumCircuit(2)
test_circuit.unitary(my_op, [0,1])

In [16]:
test_circuit.draw()

┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘

In [17]:
# Assume the HW supports the Google SYC gate. This is an underlying fsim_gate. 
# So we pass the fsim_gate function in the gate_defs,
# we pass the desired SYC parameters (theta=pi/2, phi=pi/6) in the params, we pass a label for circuit drawing, 
# and we pass the calibration fidelity of this gate on qubits 0 and 1

fid_2q = {(0,1): [0.995]} 
params = [[np.pi/2, np.pi/6]]
gate_labels = ['SYC']
gate_defs = [fsim_gate]

# All these inputs get passed to the gate decomposition pass. 
# Ignore the [1 to 54] loop (that was from our assumption of Sycamore hardware, but I believe it no longer plays a role)
# tolerance defines the fidelity accuracy we want from the decomposition. If you set the tol=0.1 or so, you can see that 
# lesser two-qubit gates are used.

pgrp = ParallelGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.01)
approx_cz = pgrp.run(test_circuit)

In [18]:
approx_cz.draw()

┌──────────────────────────┐┌──────┐┌─────────────────────────────┐»
q_0: ┤ U3(0.7162,3.5347,1.9912) ├┤1     ├┤ U3(1.5708e-05,3.896,2.2172) ├»
     ├─────────────────────────┬┘│  SYC │└─┬──────────────────────────┬┘»
q_1: ┤ U3(3.1416,6.079,2.3007) ├─┤0     ├──┤ U3(4.7842,4.5058,1.1142) ├─»
     └─────────────────────────┘ └──────┘  └──────────────────────────┘ »
«     ┌──────┐┌───────────────────────────┐
«q_0: ┤1     ├┤ U3(-0.9281,1.9121,3.4002) ├
«     │  SYC │├───────────────────────────┤
«q_1: ┤0     ├┤ U3(3.1416,1.8972,0.38331) ├
«     └──────┘└───────────────────────────┘

## Decomposition example with two circuit gates with two hardware gates

In [19]:
test_circuit = QuantumCircuit(5)
test_circuit.unitary(my_op, [2,3])
test_circuit.unitary(my_op, [3,4])

In [21]:
fid_2q = {(2,3): [0.94, #CZ fidelity
                  0.99],#XY fidelity
          (3,4): [0.93, #CZ
                  0.96]}#XY
params = [[np.pi], 
          [np.pi]]
gate_labels = ['CZ', 
               'XY']
gate_defs = [cphase_gate, 
             xy_gate]
pgrp = ParallelGateReplacementPass(gate_defs, params, gate_labels,fid_2q, [1 for _ in range(54)], tol=0.00000001)
approx = pgrp.run(test_circuit)

In [9]:
approx.draw()

»
q_0: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
q_1: ─────────────────────────────────────────────────────────────────────────»
         ┌──────────────────────────┐   ┌─────┐┌─────────────────────────────┐»
q_2: ────┤ U3(4.7124,1.7852,1.5708) ├───┤1    ├┤ U3(1.2817e-05,6.483,4.6895) ├»
      ┌──┴──────────────────────────┴─┐ │  XY │└─┬──────────────────────────┬┘»
q_3: ─┤ U3(-2.6502e-05,1.3092,2.5453) ├─┤0    ├──┤ U3(1.5708,4.5478,2.9271) ├─»
     ┌┴───────────────────────────────┴┐└─────┘  └──────────────────────────┘ »
q_4: ┤ U3(1.7814e-06,2.4564,-0.033932) ├──────────────────────────────────────»
     └─────────────────────────────────┘                                      »
«                                                   »
«q_0: ──────────────────────────────────────────────»
«                                                   »
«q_1: ──────────────────────────────────────────────»
«     ┌─────┐┌──────────────────────────────┐       »
«q_2: ┤1    ├┤ U3(1.4505e-05,6.7556,4.4046) ├───────»
«     │  XY │└┬───────────────────────────┬─┘┌─────┐»
«q_3: ┤0    ├─┤ U3(1.5708,0.84293,5.3933) ├──┤1    ├»
«     └─────┘ └───────────────────────────┘  │  CZ │»
«q_4: ───────────────────────────────────────┤0    ├»
«                                            └─────┘»
«                                      
«q_0: ─────────────────────────────────
«                                      
«q_1: ─────────────────────────────────
«                                      
«q_2: ─────────────────────────────────
«     ┌───────────────────────────────┐
«q_3: ┤ U3(4.7124,-2.1422e-05,5.4403) ├
«     └┬──────────────────────────────┤
«q_4: ─┤ U3(6.1592e-07,1.5341,5.4682) ├
«      └──────────────────────────────┘